In [126]:
""""
Agents
1. Interview researcher
   - Take the job description
   - Research and create interview questions
   - Come up with criteria for scoring the answers
2. Interview editor
    - Validates questions
    - Validates criteria for scoring
    - Cross verifies it's relevance with the job description
3. Interviewer
    - Conducts interviews
    - Scores the answers based on the set criteria
"""

'"\nAgents\n1. Interview researcher\n   - Take the job description\n   - Research and create interview questions\n   - Come up with criteria for scoring the answers\n2. Interview editor\n    - Validates questions\n    - Validates criteria for scoring\n    - Cross verifies it\'s relevance with the job description\n3. Interviewer\n    - Conducts interviews\n    - Scores the answers based on the set criteria\n'

In [127]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import load_dotenv

load_dotenv()

True

In [128]:
model = "llama3.2:1b"
ollama = LLM(model=f"ollama/{model}", base_url="http://localhost:11434")

In [129]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [130]:
interview_researcher = Agent(
    role="Interview Researcher",
    goal="Plan thorough and relevant interview questions and answer evaluation criteria based for the job of {job_title}",
    backstory="You are an expert in job role analysis and interview design. "
              "You're working on planning the interview for the job of {job_title} and your task is to extract the key responsibilities, skills, and knowledge areas from this job description: \n{job_description}\n "
              "Using this, you create a set of thoughtful, role-relevant interview questions. "
              "You also define clear, objective criteria to score answers for each question based on depth, accuracy, and relevance. "
              "Your output will be used by an Interview Editor to validate and refine the interview.",
    allow_delegation=False,
    llm=ollama,
    verbose=True,
)

In [131]:
interview_editor = Agent(
    role="Interview Editor",
    goal="Ensure that interview questions and scoring criteria aligned with the job description and communicate with the Interview Researcher",
    backstory="You are a senior editor responsible for validating the quality and relevance of interview content. "
              "You receive interview questions and answer-scoring criteria from the Interview Researcher for the job of {job_title}. "
              "Your job is to cross-check them against the original job description: ({job_description})."
              "Ensure the interview questions and answering critera accurately assess key requirements. "
              "You also verify clarity, fairness, and whether the scoring system enables consistent evaluation of candidates.",
    allow_delegation=False,
    llm=ollama,
    verbose=True,
    tools=[search_tool, scrape_tool],  # tools to validate answer criteria
)

In [132]:
interviewee = Agent(
    role="Interviewee",
    goal="Demonstrate technical expertise and problem-solving skills while showcasing professional demeanor and communication abilities",
    backstory="You are a seasoned Python developer with 7 years of experience. Your expertise includes backend development, API design, and system architecture. You have a strong foundation in Python programming, web frameworks, and database management. You are passionate about writing clean, maintainable code and have a track record of successful project delivery. You are comfortable discussing technical concepts and can explain complex ideas clearly.",

    # backstory="""
    # You are a recent college graduate with limited real-world Python experience.
    # Your knowledge is primarily from academic courses and personal projects.
    # You are familiar with basic Python syntax and some fundamental concepts,
    # but lack experience with advanced topics and professional development practices.
    # You have worked on small projects using Python, but haven't implemented
    # complex systems or worked with enterprise-level applications.
    # You are eager to learn and improve your skills, but may struggle with
    # more advanced technical questions.""",

    allow_delegation=False,
    llm=ollama,
    verbose=True,
)

In [133]:
interviewer = Agent(
    role="Interviewer",
    goal="Conduct structured interviews and score candidate responses using provided criteria",
    backstory="You are a professional interviewer conducting job interviews based on pre-approved questions. "
              "You ask the questions as they are written and actively listen to the candidate's responses. "
              "After each answer, you score it using the criteria provided by the Interview Researcher and validated by the Interview Editor. "
              "Your aim is to remain unbiased and consistent while capturing useful qualitative and quantitative data.",
    allow_delegation=False,
    llm=ollama,
    verbose=True,
    tools=[search_tool, scrape_tool],  # tools to validate answers
)


In [134]:
from pydantic import BaseModel
from enum import Enum

class DifficultyLevel(Enum):
    easy = "easy"
    medium = "medium"
    hard = "hard"

class InterviewQuestion(BaseModel):
    question: str
    difficulty: str
    answer_criteria: str

class Answer(BaseModel):
    question: str
    answer: str

class AnswerValidationOutput(BaseModel):
    question: str
    answer: str
    score_out_of_10: float
    feedback: str

In [135]:
import json
from typing import Tuple, Any

def validate_json_output(result: str) -> Tuple[bool, Any]:
    """Validate that the output is valid JSON."""
    try:
        json_data = json.loads(result)
        return (True, json_data)
    except json.JSONDecodeError:
        return (False, "Output must be valid JSON")
    except Exception as e:
        return (False, {"error": str(e), "code": "UNKNOWN_ERROR", "context": {"line": e.lineno, "column": e.colno}})


In [ ]:
research_task = Task(
    description="Research and create three questions to be asked in a job interview for the job role of {job_title}. Analyze the job description and create relevant technical and behavioral questions that assess candidate competency. For each question, define clear scoring criteria that enables objective evaluation.",
    expected_output="Set of interview questions to conduct the job interview for the job",

    agent=interview_researcher,

    output_json=InterviewQuestion,
    # output_file="interview_questions.json",  

    # guardrail=validate_json_output,
)

In [137]:
editing_task = Task(
    description="Review and validate the interview questions and scoring criteria created by the Interview Researcher. Ensure questions are clear, unbiased, and directly relevant to the job description. Verify that scoring criteria are objective, measurable, and enable consistent candidate evaluation. Provide feedback for any necessary improvements.",
    expected_output="The finalized set of interview qeustions",

    agent=interview_editor,
)

In [138]:
answer_questions_task = Task(
    description="Respond to interview questions with your skillset. Demonstrate your understanding of Python concepts, problem-solving skills, and practical experience. When appropriate, provide examples from your work or personal projects to support your answers.",

    expected_output="Answers to the interview questions",
  

    agent=interviewee,

    output_json=Answer,
    # output_file="answers.json",

    # guardrail=validate_json_output,
)

In [139]:
interview_answers_validator_task = Task(
    description="Validate the answers provided by the interviewee for the questions provided, use the answering criteria to score the responses and provide a helpful feedback based for the response",
    expected_output="A final report with the questions, answers provided by the applicant, numeric score for the answer, and the feedback.",


    agent=interviewer,

    output_json=AnswerValidationOutput,
    # output_file="final.json",

    # guardrail=validate_json_output,
)

In [143]:
interviewer_crew = Crew(
    agents=[
        interview_researcher,
        # interview_editor,
        # interviewee, interviewer,
    ],
    tasks=[
        research_task,
        # editing_task,
        # answer_questions_task,
        # interview_answers_validator_task,
    ],
    verbose=True,
)

In [144]:
job_details = {
    'job_title': "Senior Python Developer",
    'job_description': """
We are seeking a Senior Python Developer to join our engineering team. The ideal candidate will design, develop, and maintain scalable Python applications while mentoring junior developers.

Key Responsibilities:
- Design and implement high-performance, scalable Python applications
- Write clean, maintainable, and well-tested code using Python best practices
- Develop and optimize backend services and APIs
- Integrate various data storage solutions and third-party services
- Participate in code reviews and contribute to technical architecture decisions
- Mentor junior developers and promote best coding practices

Required Skills:
- 5+ years of professional Python development experience
- Strong knowledge of Python 3.x, OOP principles, and design patterns
- Experience with web frameworks (Django, FastAPI, or Flask)
- Proficiency in working with SQL and NoSQL databases
- Familiarity with RESTful APIs and microservices architecture
- Experience with version control systems (Git)
- Strong understanding of software testing principles
- Knowledge of containerization (Docker) and CI/CD pipelines

Nice to Have:
- Experience with async Python programming
- Knowledge of cloud platforms (AWS, GCP, or Azure)
- Contributions to open-source projects
- Experience with Python data science libraries
- Background in Agile development methodologies

We offer a collaborative environment, opportunities for professional growth, and the chance to work on challenging projects using cutting-edge technologies."""
}

In [145]:
result = interviewer_crew.kickoff(inputs=job_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e67da0f1-e638-4667-8db6-0f213b9ee51f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a536e8e0-e154-45a0-af6a-050183d1c443
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a536e8e0-e154-45a0-af6a-050183d1c443
       Status: Executing Task...
    └── 🤖 Agent: Interview Researcher
            Status: In Progress

# Agent: Interview Researcher
## Task: Research and create three questions to be asked in a job interview for the job role of Senior Python Developer. Analyze the job description and create relevant technical and behavioral questions that assess candidate competency. For each question, define clear scoring criteria that enables objective evaluation.


🚀 Crew: crew
└── 📋 Task: a536e8e0-e154-45a0-af6a-050183d1c443
       Status: Executing Task...
    └── 🤖 Agent: Interview Researcher
            Status: In Progress
        └── 🧠 Thinking...

...

/Users/sonubardai/Desktop/AgentAi/interview_ai/venv/lib/python3.11/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Failed to convert text into JSON, error: 3 validation errors for InterviewQuestion
question
  Field required [type=missing, input_value={'properties': {'keyRespo... Python applications?'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
difficulty
  Field required [type=missing, input_value={'properties': {'keyRespo... Python applications?'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
answer_criteria
  Field required [type=missing, input_value={'properties': {'keyRespo... Python applications?'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing. Using raw output instead.


🚀 Crew: crew
└── 📋 Task: a536e8e0-e154-45a0-af6a-050183d1c443
       Assigned to: Interview Researcher
       Status: ✅ Completed
    └── 🤖 Agent: Interview Researcher
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a536e8e0-e154-45a0-af6a-050183d1c443                                                                     │
│  Agent: Interview Researcher                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e67da0f1-e638-4667-8db6-0f213b9ee51f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [147]:
# result.tasks_output[0]
for item in result.tasks_output:
    print(f"OUTPUT FROM {item.agent}: ")
    print(item.raw)
    print("\n----------------------------\n")
result.token_usage

OUTPUT FROM Interview Researcher: 
### Question 1: What are the key responsibilities of designing and implementing high-performance, scalable Python applications?

- **Difficulty**: Medium-High
- **Answer Criteria**:
    + Depth (10 points): Can the candidate describe specific technologies and frameworks used for building scalable Python applications?
    + Accuracy (20 points): Are they confident in their ability to design and implement high-performance applications using Python best practices?
    + Relevance (30 points): Is the answer relevant to the role and industry standard practices?

### Question 2: How do you ensure code maintainability, testability, and quality in your Python applications?

- **Difficulty**: Medium
- **Answer Criteria**:
    + Depth (10 points): Can the candidate explain their approach to writing clean, modular, and reusable code?
    + Accuracy (20 points): Are they confident in their ability to write tests for their applications using various testing framew

UsageMetrics(total_tokens=3838, prompt_tokens=2128, cached_prompt_tokens=0, completion_tokens=1710, successful_requests=3)